In [1]:
import getpass
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


# Simpler classification control 
"""
class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")

"""


# More fine-grained control
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive.",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )


# LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(
    Classification
)

In [6]:
#inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response

Classification(sentiment='sad', aggressiveness=5, language='spanish')

In [17]:
prompt

ChatPromptValue(messages=[HumanMessage(content="\nExtract the desired information from the following passage.\n\nOnly extract the properties mentioned in the 'Classification' function.\n\nPassage:\nEstoy muy enojado con vos! Te voy a dar tu merecido!\n", additional_kwargs={}, response_metadata={})])

In [19]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

response.model_dump()

{'sentiment': 'sad', 'aggressiveness': 5, 'language': 'spanish'}